In [12]:
%stop_session

Stopping session: 75f46a66-8522-469a-877e-a0c175df65ae
Stopped session.


In [18]:
%glue_ray
%additional_python_modules transformers,torch
%min_workers 25
%number_of_workers 25
%object_memory_worker 10

Previous Job type: glueray
Setting new Job type to glueray
Additional python modules to be included:
transformers
torch
Setting new minimum workers: 25
Previous number of workers: 25
Setting new number of workers to: 25
Setting new object_memory_worker to: 10 %


In [1]:
import sys

print(sys.version)

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518
Trying to create a Glue session for the kernel.
Worker Type: Z.2X
Number of Workers: 25
Session ID: e5b7c6bc-9dae-4050-b655-c67696437170
Job Type: glueray
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--auto-scaling-ray-min-workers 25
--additional-python-modules transformers,torch
--object_store_memory_worker 10
Waiting for session e5b7c6bc-9dae-4050-b655-c67696437170 to get into ready status...
Session e5b7c6bc-9dae-4050-b655-c67696437170 has been created.
3.9.14 (main, Oct 22 2022, 00:08:38) 
[GCC 7.3.1 20180712 (Red Hat 7.3.1-15)]


In [2]:
import ray

df = ray.data.read_parquet("s3://dsoaws/parquet/") \
                          .drop_columns(cols=["customer_id", "review_id", "product_id", "product_parent", "product_title", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_date", "year", "product_category", "marketplace"])

df.schema

<bound method Dataset.schema of Dataset(num_blocks=430, num_rows=160796570, schema={review_body: object})>


Read->Map_Batches: 100%|##########| 430/430 [00:24<00:00, 17.45it/s]


Stopping session: e5b7c6bc-9dae-4050-b655-c67696437170


In [3]:
df.count()

160796570


In [ ]:
os.environ['TRANSFORMERS_CACHE'] = '/tmp/huggingface/'

import transformers
from transformers import AutoTokenizer

model_checkpoint = "bigscience/bloom-560m"    

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, cache_dir='/tmp/huggingface/')

Execution Interrupted. Attempting to cancel the statement (statement_id=476105330)


In [ ]:
df_tokenized = df.limit(2).map_batches(lambda batch: tokenizer(batch))
df_tokenized

In [ ]:
df_tokenized.count()

In [ ]:
df_train, df_validation = df_tokenized.train_test_split(test_size=0.2)

print(f'Train dataset size is {df_train.count()} and validation dataset size is {df_validation.count()}.')

In [ ]:
df_tokenized.show()

In [ ]:
# num_workers = 100
# df_train.repartition(num_workers)
# df_validation.repartition(num_workers)

In [ ]:
# !aws s3 rm --recursive s3://dsoaws/data-bloom/

In [ ]:
df_train.write_json('s3://dsoaws/data-bloom/train', try_create_dir=True) #, cache_dir='/tmp/huggingface')
df_validation.write_json('s3://dsoaws/data-bloom/validation', try_create_dir=True) # , cache_dir='/tmp/huggingface')